In [1]:
import gurobipy as gb
import numpy as np
import csv

In [2]:
NUMDISTRICTS = 9

# basic voter data as numpy arrays - POPULATION, DEMOCRATS, REPUBLICANS, INDEPENDENTS
# each town is assigned an index in the idx dict; e.g. to get index of BEDFORD use idx['BEDFORD']
# e.g. to get population of BEDFORD use POP[idx['BEDFORD']]
with open("Voter_Data_MA_CSV_Version_2.csv", 'r') as votefile:
    votereader = csv.reader(votefile, delimiter=',')
    next(votereader)
    TOWNS = []
    POP = []
    DEMS = []
    REPS = []
    INDS = []
    idx = {}
    i = 0
    for row in votereader:
        town = row[0]
        TOWNS.append(town)
        idx[town] = i
        i += 1
        POP.append(int(row[-1]))
        DEMS.append(int(row[-3]))
        REPS.append(int(row[-2]))
        INDS.append(POP[-1] - DEMS[-1] - REPS[-1])
    POP = np.array(POP)
    DEMS = np.array(DEMS)
    REPS = np.array(REPS)
    INDS = np.array(INDS)

# pairwise distance data, as a numpy array
# use idx dict to get indices
with open("dist_pairs.csv", 'r') as distfile:
    # pairwise dist
    distreader = csv.reader(distfile, delimiter=',')
    DIST = np.zeros((i,i))
    for row in distreader:
        DIST[idx[row[0]]][idx[row[1]]] = float(row[-1])

# pairwise adjacency matrix (1/0 for adjacent/not adj)
# use idx dict to get indices
with open ('Adjacencies.csv', 'r') as adjfile:
    adjreader = csv.reader(adjfile, delimiter=',')
    ADJ = np.zeros((i,i))
    for row in adjreader:
        i1 = idx[row[0]]
        for t2 in row[1:]:
            if not t2 == '':
                ADJ[i1][idx[t2]] = 1

In [3]:
# calculate adjusted vote distributions so that each party has 50% vote share
TOTAL_DEMS = float(sum(DEMS))
TOTAL_REPS = float(sum(REPS))
TOTAL_INDS = float(sum(INDS))
TOTAL_POP = sum(POP)
p = (TOTAL_REPS + TOTAL_INDS - TOTAL_DEMS)/(2*TOTAL_INDS)
ADJUSTED_DEMS = (DEMS + p * INDS).astype(int)
ADJUSTED_REPS = (REPS + (1-p) * INDS).astype(int)

In [4]:
# init model
m = gb.Model()

# big-M (for population of state)
M = 10**6

# coeff of radius term in objective
l = 10**(-4)

# min and max population for each district
MINPOP = 720000
MAXPOP = 735000

# init variables
print('INITIALIZING VARIABLES')
# dw_i - if democrat wins district i (0/1)
# rw_i - if republican wins district i (0/1)
# PB - positive difference between D and R
# z_t^i - if town t in district i (0/1)
# r_i - max dist between towns in district i, 0<=i<=9 (pos float)
PB = m.addVar(name='PB', vtype=gb.GRB.INTEGER)
z = [[] for t in range(len(idx))]
for t, zt in enumerate(z):
    for i in range(NUMDISTRICTS):
        zt.append(m.addVar(name='z_'+str(t)+'('+str(i)+')', vtype=gb.GRB.BINARY))    
r = []
dw = []
rw = []
for i in range(NUMDISTRICTS):
    r.append(m.addVar(name='r'+str(i)))
    dw.append(m.addVar(name='dw'+str(i), vtype=gb.GRB.BINARY))
    rw.append(m.addVar(name='rw'+str(i), vtype=gb.GRB.BINARY))
m.update()

# init constraints
print('INITIALIZING CONSTRAINTS')
# each town is in exactly one district
# sum_i z_t^i = 1
for t in range(len(TOWNS)):
    m.addConstr(sum(z[t]), sense=gb.GRB.EQUAL, rhs=1., name='towndist'+str(t))
    
# constraints to show which party won a district
# dw_i + rw_i = 1 for all district i
# M * dw_i - sum_t(z_t^i*(DEMS_i - REPS_i)) >= 0
# M * rw_i - sum_t(z_t^i*(REPS_i - DEMS_i)) > 0
for i in range(NUMDISTRICTS):
    m.addConstr(dw[i] + rw[i], sense=gb.GRB.EQUAL, rhs=1., name='onewinner'+str(i))
    m.addConstr(M*dw[i] - sum([z[t][i]*(ADJUSTED_DEMS[t] - ADJUSTED_REPS[t]) for t in range(len(TOWNS))]), sense=gb.GRB.GREATER_EQUAL, rhs=0., name='demwin'+str(i))
    m.addConstr(M*rw[i] - sum([z[t][i]*(ADJUSTED_REPS[t] - ADJUSTED_DEMS[t]) for t in range(len(TOWNS))]), sense=gb.GRB.GREATER_EQUAL, rhs=0., name='repwin'+str(i))

# count number of dem/rep districts, and partisan bias
# PB >= D-R, PB >= R-D
m.addConstr(PB - sum(dw) + sum(rw), sense=gb.GRB.GREATER_EQUAL, rhs=0., name='PB1')
m.addConstr(PB - sum(rw) + sum(dw), sense=gb.GRB.GREATER_EQUAL, rhs=0., name='PB2')
    
# diameter r_i of a district is greater than every pairwise distance between towns in the district
# r_i >= dist_st * (z_s^i + z_t^i - 2) for all districts i, towns s t
for i in range(NUMDISTRICTS):
    for s in range(len(TOWNS)):
        for t in range(len(TOWNS)):
            m.addConstr(r[i] - DIST[s][t] * (z[s][i] + z[t][i] - 1.), sense=gb.GRB.GREATER_EQUAL, rhs=0., name="r"+str(i)+'_'+str(s)+'-'+str(t))
            
# district population constraint:
# MINPOP <= sum_t z_t^i * POP_i <= MAXPOP for all districts i
for i in range(NUMDISTRICTS):
    m.addConstr(sum([z[t][i]*POP[t] for t in range(len(TOWNS))]), sense=gb.GRB.LESS_EQUAL, rhs=MAXPOP, name='maxpop'+str(i))
    m.addConstr(sum([z[t][i]*POP[t] for t in range(len(TOWNS))]), sense=gb.GRB.GREATER_EQUAL, rhs=MINPOP, name='minpop'+str(i))
    
# objective
print('INITIALIZING OBJECTIVE')
# min PB + sum_i r_i (districts i)
m.update()
m.setObjective(PB + l*sum(r), gb.GRB.MINIMIZE)
print('SOLVING')
m.optimize()

Academic license - for non-commercial use only
INITIALIZING VARIABLES
INITIALIZING CONSTRAINTS
INITIALIZING OBJECTIVE
SOLVING
Optimize a model with 1109209 rows, 3189 columns and 3323348 nonzeros
Variable types: 9 continuous, 3180 integer (3177 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e-04, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+05]
Presolve removed 559143 rows and 9 columns (presolve time = 5s) ...
Presolve removed 559144 rows and 10 columns (presolve time = 10s) ...
Presolve removed 559144 rows and 10 columns
Presolve time: 14.25s
Presolved: 550065 rows, 3179 columns, 1664834 nonzeros
Variable types: 9 continuous, 3170 integer (3168 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   8.429925e+04     21s
   12110    1.9317693e+00   0.000000e+00   2.564511e+06     26s
   20381    4.0979132e-03   7.370484e-03   0.000000e+00  

   440   301    1.01400    6 1049    1.09111    1.00335  8.04%  4927 19452s
   445   306    1.01472    9 1191    1.09111    1.00335  8.04%  5030 20237s
   450   307    1.01003    9 1796    1.09111    1.00335  8.04%  5096 29638s

Cutting planes:
  Gomory: 102
  MIR: 404

Explored 452 nodes (2377708 simplex iterations) in 29639.06 seconds
Thread count was 4 (of 4 available processors)

Solution count 10: 1.09111 1.09482 1.10016 ... 1.14499

Solve interrupted
Best objective 1.091109515191e+00, best bound 1.003350354366e+00, gap 8.0431%


In [6]:
print([ri.x for ri in r])
print(sum([ri.x for ri in r]) * 10**(-4))

[109.34014584920783, 92.89297263136113, 97.3225137038076, 147.76656047904544, 4.445141393821245, 76.76506414829142, 219.50969328894897, 70.67126680306023, 92.38179360907824]
0.09110951519066222


In [8]:
with open("PB_testrun.csv", 'w') as f:
    dwriter = csv.writer(f, delimiter=',')
    for i in range(NUMDISTRICTS):
        towns = []
        for t in range(len(TOWNS)):
            if z[t][i].x == 0:
                towns.append(TOWNS[t])
        print('********************************')
        print('DISTRICT '+str(i))
        print(towns)
        dwriter.writerow(towns)

********************************
DISTRICT 0
['ADAMS', 'ALFORD', 'BECKET', 'CHESHIRE', 'CLARKSBURG', 'DALTON', 'EGREMONT', 'FLORIDA', 'GREAT BARRINGTON', 'HANCOCK', 'HINSDALE', 'LANESBOROUGH', 'LEE', 'LENOX', 'MONTEREY', 'MOUNT WASHINGTON', 'NEW ASHFORD', 'NEW MARLBOROUGH', 'NORTH ADAMS', 'OTIS', 'PERU', 'PITTSFIELD', 'RICHMOND', 'SANDISFIELD', 'SAVOY', 'SHEFFIELD', 'STOCKBRIDGE', 'TYRINGHAM', 'WASHINGTON', 'WEST STOCKBRIDGE', 'WILLIAMSTOWN', 'WINDSOR', 'ATTLEBORO', 'DIGHTON', 'EASTON', 'MANSFIELD', 'NORTH ATTLEBOROUGH', 'NORTON', 'RAYNHAM', 'REHOBOTH', 'SEEKONK', 'SWANSEA', 'TAUNTON', 'WESTPORT', 'AMESBURY', 'ANDOVER', 'BEVERLY', 'BOXFORD', 'DANVERS', 'ESSEX', 'GEORGETOWN', 'GLOUCESTER', 'GROVELAND', 'HAMILTON', 'HAVERHILL', 'IPSWICH', 'LAWRENCE', 'LYNN', 'LYNNFIELD', 'MANCHESTER-BY-THE-SEA', 'MARBLEHEAD', 'MERRIMAC', 'METHUEN', 'MIDDLETON', 'NAHANT', 'NEWBURY', 'NEWBURYPORT', 'NORTH ANDOVER', 'PEABODY', 'ROCKPORT', 'ROWLEY', 'SALEM', 'SALISBURY', 'SAUGUS', 'SWAMPSCOTT', 'TOPSFIELD', '